Imports

In [78]:
import cv2
import numpy as np
import pickle
import cv2
import math

Loading Models


In [79]:
binary_meNotme_model = pickle.load(open('Sameed_unknown_classifier.pkl', 'rb'))
multi_meNotme_model = pickle.load(open('Sameed_Arham_Unknown_classifier.pkl', 'rb'))
age_model = pickle.load(open('Age_regression_ridge.pkl', 'rb'))


Face Detection in Frame

In [80]:
# Binds face in the frame with a box
# If face is detected it returns an image with the face bound, and the pixel area inside the box, excluding the actual boundary
# If there are multiple faces in frame, it chooses the one having the greatest area in frame

def faceDetect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(64, 64)
    )
    
    if(len(faces) > 1):
        cur_max = 0
        face = 0

        for (x, y, w, h) in faces:
            length = math.sqrt((x - (x+w))**2 + (y - (y+h))**2 )
            
            if length > cur_max:
                cur_max = length
                face = [x,y,w,h]

        faces = np.array([face])

    for (x, y, w, h) in faces:
    
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # Draw a rectangle around the detected area (face)

        roi_color = image[y+2:y+2 + h-3, x+2:x+2 + w-3] # The pixel area containing the face

        return True,[image,roi_color]
    
    return False,[]

Camera Predictions

In [83]:
# Starts the camera, provides each frame to the faceDetect() function
# If a face is detected it prints the Identity(Sameed or Unknown) and Age using Linear and SGD regression at the bottom of the screen
# If no face detected, it prints appropriately
# Use esc to exit

cv2.destroyAllWindows()
cam = cv2.VideoCapture(0)

while True:

    check, frame = cam.read()
    frame = cv2.flip(frame,1)

    
    frame = cv2.copyMakeBorder(frame,1,250,1,280,cv2.BORDER_CONSTANT,value=0)



    cv2.imshow('video', frame)
    detected,bound_frame = faceDetect(frame)


    if(detected):

        cv2.putText(frame,"Binary Who ->",(20,530),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        cv2.putText(frame,"Multi Who  ->",(20,570),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        cv2.putText(frame,"Age        ->",(20,670),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)

        
        frame = bound_frame[0]
        cropped_frame = bound_frame[1]

        img_resize = cv2.resize(cropped_frame, (64,64)) # The classifier was trained on 64x64 images
        img_resize_32 = cv2.resize(cropped_frame, (32,32)) # The classifier was trained on 64x64 images


        cur_image = img_resize.flatten()
        cur_image_32 = img_resize_32.flatten()
        

        binary_pred = binary_meNotme_model.predict([cur_image])
        multi_pred = multi_meNotme_model.predict([cur_image])
        pred_proba = multi_meNotme_model.predict_proba([cur_image])[0]


        age_pred = age_model.predict([cur_image_32])

        
        cv2.putText(frame,str(age_pred),(270,670),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        cv2.putText(frame,"(1. Unknown)",(20,630),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        cv2.putText(frame,"(2. Sameed)",(250,630),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        cv2.putText(frame,"(3. Arham)",(500,630),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)



        if(binary_pred == 0):
            cv2.putText(frame,"Unknown",(270,530),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            # cv2.imshow('video', frame)

        else:
            cv2.putText(frame,"Sameed",(270,530),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            # cv2.imshow('video', frame)

        if(multi_pred == 0):
            cv2.putText(frame,"Unknown",(270,570),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            cv2.putText(frame,str(pred_proba),(20,600),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)

        
        if(multi_pred == 1):
            cv2.putText(frame,"Sameed",(270,570),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            cv2.putText(frame,str(pred_proba),(20,600),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)

        if(multi_pred == 2):
            cv2.putText(frame,"Arham",(270,570),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            cv2.putText(frame,str(pred_proba),(20,600),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)


    else:
        cv2.putText(frame,"Face Not Detected",(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        # cv2.imshow('video', frame)

    cv2.imshow('video', frame)


    key = cv2.waitKey(1)
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()